# Data Engineering Activity

## Available Data
I have access to the following datasets:

- **dim_Organisation**: Contains school records for various Academies within the same school group.
- **dim_Student**: Includes core information about each student.
- **dim_StudentExtended**: Provides additional details for each student.
- **fact_AttendanceSession**: Records daily attendance for each student, divided into two sessions: AM and PM. A student's attendance percentage is calculated as the sum of `is_present` divided by the sum of `is_possible`.
- **dim_Date**: A CSV file serving as a date dimension, with `FullDate` as the date column.

## Activity Brief

My task is to create summary analytics to assist the Data Analyst in building an Attendance Report. I will follow these steps:

1. **Data Loading**
    - I will create a Jupyter Notebook to load the parquet files and the CSV into DataFrames.
    - I will use an environment of my choice (e.g., Databricks Community Edition, VSCode with Jupyter Extension).

2. **Data Exploration**
    - I will determine the key relationships between the tables.

3. **Summary Table Creation**
    - I will produce a single summary table that includes the attendance percentage for each school on a weekly basis, categorized by the student's Year Group.
    - I will use my judgement to select the appropriate columns to include.

4. **Analytics**
    - I will print or display summary statistics about the data within the Notebook.

5. **Exporting Results**
    - I will write the summary table to `fact_AttendanceSummary` in Parquet format.




### Data Engineering Activity - Outline of Approach


I will use **PySpark**, given the large size of the dataset, which contains approximately 16 million rows. I will follow these steps to complete the task:

1. **Import Libraries**: Load the necessary PySpark libraries.
2. **Write Methods**: write methods of tasks which I will repeatedly use
3. **View the data**: Import the dataset into a PySpark DataFrame. Conduct an initial exploration to understand the structure, data types, and key attributes of the dataset.  
4. **Join the data**: Merge the tables to create a single DataFrame containing all relevant information.  
5. **Inspect Distinct Values**: Check the unique values in each column to identify any inconsistencies or missing values. 
6. **Select Relevant Columns**: Choose the necessary columns for the analysis.
7. **Data Integrity Check**: Ensure data integrity by feature engineering a key to ensure each pupil has two entrices for each day. 
8. **Calculate Attendance Percentage**: Calculate the attendance percentage for each student based on the attendance records.Create a summary table containing the attendance percentage for each school on a weekly basis, grouped by the **Year Group** of the students.  
9. **Investigate Null Values**: Analyse the distribution of null values in the dataset and decide on the appropriate handling strategy.
10. **Write the Summary Table**: Display key summary statistics in the Notebook, including metrics such as the mean, median, minimum, maximum, and standard deviation of attendance percentages.  Export the summary table to `fact_AttendanceSummary` in **Parquet** format for further analysis and reporting.
11. **Notes for Data Analyst**: Provide additional notes or insights that may be useful for the Data Analyst when building the Attendance Report.

Each section is labelled accordingly with a markdown header for clarity and ease of navigation.

In [1]:
# SparkSession is the entry point to Spark SQL

from pyspark.sql import SparkSession 

# Create a SparkSession and set memory for the driver
spark = SparkSession.builder \
    .appName("MyApp") \
    .config("spark.driver.memory", "8g") \
    .getOrCreate()


In [2]:
# Load CSV files into Spark DataFrames and infer schema to avoid specifying it manually
df_date_spark = spark.read \
    .option("header", True) \
    .option("inferSchema", True) \
    .csv("data/dim_Date.csv")

# Load parquet files into Spark DataFrames 
df_attendancesessions_spark = spark.read.parquet("data/fact_AttendanceSession")
df_organisation_spark = spark.read.parquet("data/dim_Organisation")
df_student_spark = spark.read.parquet("data/dim_Student")
df_studentextended_spark = spark.read.parquet("data/dim_StudentExtended")


#### 1. Import Libraries 

In [3]:
from pyspark.sql import DataFrame
from pyspark.sql import functions as F
from pyspark.sql.functions import countDistinct
from functools import reduce



#### 2. Methods

In [4]:


def show_df_missing_breakdown(df: DataFrame) -> None:
    """
    Prints:
      - The DataFrame size (rows, columns)
      - For each column:
        * number of NULLs
        * number of empty strings
        * number of 'NA' / 'NaN' (case-insensitive) as strings
        * number of numeric NaNs (for numeric columns)
        * total missing (sum of above)
        * percentage missing
    """
    total_rows = df.count()
    total_cols = len(df.columns)

    # Prepare expressions for counting different "missing" types for each column
    agg_exprs = []
    for field in df.schema.fields:
        col_name = field.name
        # Check if column is numeric (so we can safely use F.isnan)
        is_numeric = field.dataType.typeName() in (
            "double", "float", "decimal", 
            "integer", "long", "short", "byte"
        )

        # Count NULLs
        null_count_expr = F.sum(
            F.when(F.col(col_name).isNull(), 1).otherwise(0)
        ).alias(col_name + "_nullCount")

        # Count empty strings
        empty_count_expr = F.sum(
            F.when(F.col(col_name).cast("string") == "", 1).otherwise(0)
        ).alias(col_name + "_emptyCount")

        # Count string 'NA' or 'NaN' (case-insensitive)
        na_str_expr = F.sum(
            F.when(
                F.upper(F.col(col_name).cast("string")).isin("NA", "NAN"), 
                1
            ).otherwise(0)
        ).alias(col_name + "_naStrCount")

        # Count numeric NaN (only for numeric columns)
        if is_numeric:
            nan_numeric_expr = F.sum(
                F.when(F.isnan(F.col(col_name)), 1).otherwise(0)
            ).alias(col_name + "_nanNumericCount")
        else:
            # For non-numeric columns, this will always be 0
            nan_numeric_expr = F.lit(0).alias(col_name + "_nanNumericCount")

        # Collect all expressions
        agg_exprs.extend([
            null_count_expr, empty_count_expr, na_str_expr, nan_numeric_expr
        ])

    # Perform a single pass to get all missing counts
    agg_df = df.select(agg_exprs)
    result_row = agg_df.collect()[0].asDict()  # single row with all counts

    # Print header
    print(f"DataFrame has {total_rows} rows and {total_cols} columns.\n")
    print(
        "Column                             "
        "Null  EmptyStr  NA/NaNStr  NumericNaN  TotalMissing  %Missing"
    )
    print("-" * 70)

    # Loop over columns and print breakdown
    for field in df.schema.fields:
        c = field.name
        null_count = result_row[c + "_nullCount"]
        empty_count = result_row[c + "_emptyCount"]
        na_str_count = result_row[c + "_naStrCount"]
        nan_numeric_count = result_row[c + "_nanNumericCount"]

        total_missing = null_count + empty_count + na_str_count + nan_numeric_count
        pct_missing = (total_missing / total_rows * 100) if total_rows else 0.0

        print(
            f"{c:34s}"
            f"{null_count:5d}"
            f"{empty_count:10d}"
            f"{na_str_count:10d}"
            f"{nan_numeric_count:12d}"
            f"{total_missing:13d}"
            f"{pct_missing:10.2f}%"
        )


In [5]:

def show_distinct_counts(df: DataFrame, top_n: int = 20) -> None:
    """
    Displays the number of distinct values in each column of the DataFrame
    and lists the top_n columns with the highest distinct counts.
    
    Additionally, creates and displays a DataFrame containing all columns with their distinct counts.
    
    Parameters:
    df (DataFrame): The Spark DataFrame to analyze.
    top_n (int): The number of top columns to display based on distinct counts.
    """
    # Calculate distinct counts for each column
    distinct_counts = df.agg(*[countDistinct(c).alias(c) for c in df.columns]).collect()[0].asDict()
    
    # Sort columns by distinct count in descending order
    sorted_counts = sorted(distinct_counts.items(), key=lambda x: x[1], reverse=True)
    
    # Display the top_n columns
    print(f"{'Column':34s} {'Distinct Count'}")
    print("-" * 50)
    for col, cnt in sorted_counts[:top_n]:
        print(f"{col:34s} {cnt}")
    
    # Create a DataFrame of all distinct counts
    df_distinct_counts = spark.createDataFrame(sorted_counts, ["Column", "Distinct_Count"])
    
    # Show the DataFrame of distinct counts
    print("\nAll Column Distinct Counts:")
    df_distinct_counts.show(truncate=False)

In [6]:
from pyspark.sql import SparkSession, DataFrame, functions as F

def show_distinct_counts_approx(df: DataFrame, top_n: int = 20, rsd: float = 0.05) -> None:
    """
    Displays the approximate number of distinct values in each column of the DataFrame
    and lists the top_n columns with the highest distinct counts.

    Additionally, creates and displays a DataFrame containing all columns with 
    their approximate distinct counts, but only shows the top_n rows to reduce 
    the chance of memory/network issues.

    Parameters:
    -----------
    df : DataFrame
        The Spark DataFrame to analyze.
    top_n : int
        The number of top columns to display based on distinct counts.
    rsd : float
        Relative Standard Deviation for approx_count_distinct. 
        Lower = more accurate but more memory usage. Typical default is 0.05.
    """

    # Build a list of approx_count_distinct expressions for each column
    approx_exprs = [
        F.approx_count_distinct(F.col(c), rsd=rsd).alias(c)
        for c in df.columns
    ]

    # Collect the single row of approximate distinct counts as a dict
    #  e.g. {'colA': 123, 'colB': 999, ...}
    approx_counts_row = df.agg(*approx_exprs).collect()[0].asDict()

    # Convert that dict into a list of (column, distinct_count) tuples and sort
    sorted_counts = sorted(approx_counts_row.items(), key=lambda x: x[1], reverse=True)

    # Print header
    print(f"{'Column':34s} {'Approx Distinct Count'}")
    print("-" * 60)

    # Show only the top_n columns in console
    for col_name, cnt in sorted_counts[:top_n]:
        print(f"{col_name:34s} {cnt}")

    # Create a small DataFrame from the sorted counts
    # Each row: (column_name, approx_distinct_count)
    spark = SparkSession.builder.getOrCreate()
    df_approx_counts = spark.createDataFrame(
        sorted_counts, ["Column", "ApproxDistinctCount"]
    )

    # Show only the top_n rows, so we don't blow up memory
    print("\nAll Column Approx Distinct Counts (showing top_n only):")
    df_approx_counts.limit(top_n).show(truncate=False)


#### 3. View the Data 

In [7]:
# Show the first 20 rows of the DataFrames
df_studentextended_spark.show()

+---------------+--------------+------------+--------------+-----------------+-------------------+--------------------------+-----------+-----------------------+----------+------------------------------+-------------------------------------+-------------+---------------------+--------------------+--------------------+----------+---------------+--------------------+--------------------+----------+--------+--------------------+--------------------+--------------------+
|      Ethnicity|Ethnicity_Code|Ever_In_Care|First_Language|Free_School_Meals|Free_School_Meals_6|Gifted_And_Talented_Status|In_LEA_Care|Pupil_Premium_Indicator|SEN_Status|English_As_Additional_Language|English_As_Additional_Language_Status|Child_In_Need|Child_Protection_Plan|    Enrolment_Status|  studentextendedkey|Year_Group|Current_NC_Year|      Admission_Date|        Leaving_Date|Is_Current|Postcode|     organisationkey|          studentkey|        partitionkey|
+---------------+--------------+------------+-----------

I will the missing value method defined earlier to provide information on size and missing values in the data.

In [8]:
show_df_missing_breakdown(df_studentextended_spark)

DataFrame has 16937 rows and 25 columns.

Column                             Null  EmptyStr  NA/NaNStr  NumericNaN  TotalMissing  %Missing
----------------------------------------------------------------------
Ethnicity                             0         0         0           0            0      0.00%
Ethnicity_Code                        0         0         0           0            0      0.00%
Ever_In_Care                          0         0         0           0            0      0.00%
First_Language                        0         0         0           0            0      0.00%
Free_School_Meals                     0         0         0           0            0      0.00%
Free_School_Meals_6                   0         0         0           0            0      0.00%
Gifted_And_Talented_Status            0         0         0           0            0      0.00%
In_LEA_Care                           0         0         0           0            0      0.00%
Pupil_Premium_Indicato

Initial obersevations:
 - Three keys are available in the data: student_id, organisation_id, date
 - Dateframe df includes students who are current and have left the school
 - National currlculum year (age based) and year group are available - are NULL if the student has left the school

In [9]:
df_student_spark.show()

+--------+--------------+-------------+-------+------------+------+------+-------------+--------------------+--------------------+---+--------------------+
|Forename|Legal_Forename|Legal_Surname|Surname|Middle_Names|   Sex|Gender|Date_Of_Birth|     organisationkey|          studentkey|UPN|        partitionkey|
+--------+--------------+-------------+-------+------------+------+------+-------------+--------------------+--------------------+---+--------------------+
| Deborah|        XXXXXX|       XXXXXX|   Wong|      XXXXXX|FEMALE|  None|   2000-01-01|02ef2e04-5a06-4f1...|0002c6c1-11bd-4a4...|  0|02ef2e04-5a06-4f1...|
|Michelle|        XXXXXX|       XXXXXX|  Glenn|      XXXXXX|FEMALE|  None|   2000-01-01|02ef2e04-5a06-4f1...|002e1bf1-f48f-4a8...|  7|02ef2e04-5a06-4f1...|
|  Alyssa|        XXXXXX|       XXXXXX|Schmidt|      XXXXXX|  MALE|  None|   2000-01-01|02ef2e04-5a06-4f1...|0050d3ad-83b5-423...| 14|02ef2e04-5a06-4f1...|
|    Juan|        XXXXXX|       XXXXXX|Ramirez|      XXXXXX|FEMA

In [10]:
show_df_missing_breakdown(df_student_spark)

DataFrame has 16937 rows and 12 columns.

Column                             Null  EmptyStr  NA/NaNStr  NumericNaN  TotalMissing  %Missing
----------------------------------------------------------------------
Forename                              0         0         0           0            0      0.00%
Legal_Forename                        0         0         0           0            0      0.00%
Legal_Surname                         0         0         0           0            0      0.00%
Surname                               0         0         0           0            0      0.00%
Middle_Names                          0         0         0           0            0      0.00%
Sex                                   0         0         0           0            0      0.00%
Gender                                0         0         0           0            0      0.00%
Date_Of_Birth                         0         0         0           0            0      0.00%
organisationkey       

Initial observations:
- XXXXX used to keep data confidential - 
- gender and sex are the same - Gender can be dropped as it contains XXXXX
- Same with legal_forename and forename, legal_surname and surname



In [11]:
df_organisation_spark.show()

+-----------------+--------------------+-------+-----------------+--------------------+----------+-----+-------------------+------------+---+--------------------+
|Organisation_Name|Establishment_Number|LA_Code|Organisation_Type|     organisationkey|addresskey|UKPRN|Organisation_Status|last_updated|URN|        partitionkey|
+-----------------+--------------------+-------+-----------------+--------------------+----------+-----+-------------------+------------+---+--------------------+
|        Academy 4|              XXXXXX| XXXXXX|        SECONDARY|02ef2e04-5a06-4f1...|          |     |             Active|            |  3|02ef2e04-5a06-4f1...|
|        Academy 9|              XXXXXX| XXXXXX|      ALL THROUGH|068cf4c6-2526-430...|          |     |             Active|            |  8|068cf4c6-2526-430...|
|        Academy 7|              XXXXXX| XXXXXX|        SECONDARY|2d9ba2ce-d6e9-49b...|          |     |             Active|            |  6|2d9ba2ce-d6e9-49b...|
|        Academy 2|   

In [12]:
show_df_missing_breakdown(df_organisation_spark)

DataFrame has 9 rows and 11 columns.

Column                             Null  EmptyStr  NA/NaNStr  NumericNaN  TotalMissing  %Missing
----------------------------------------------------------------------
Organisation_Name                     0         0         0           0            0      0.00%
Establishment_Number                  0         0         0           0            0      0.00%
LA_Code                               0         0         0           0            0      0.00%
Organisation_Type                     0         0         0           0            0      0.00%
organisationkey                       0         0         0           0            0      0.00%
addresskey                            0         9         0           0            9    100.00%
UKPRN                                 0         9         0           0            9    100.00%
Organisation_Status                   0         0         0           0            0      0.00%
last_updated              

Initial observations:
- Assumption XXXX used to keep data confidential - e.g. LA Code; these columns can be kept for use of the script with other data

In [13]:
df_attendancesessions_spark.show()

+----------+----+-------+--------------------+------+---------+-----------+---------+---------+----------+-----+-----------+----------+-------------+--------------------+--------------------+--------------------+
|      Date|Mark|Session|attendancesessionkey|is_aea|is_attend|is_auth_abs|is_late_L|is_late_U|is_missing|is_nr|is_possible|is_present|is_unauth_abs|     organisationkey|          studentkey|        partitionkey|
+----------+----+-------+--------------------+------+---------+-----------+---------+---------+----------+-----+-----------+----------+-------------+--------------------+--------------------+--------------------+
|2023-11-13|   /|     AM|162ff625-c9d6-49e...|   0.0|      1.0|        0.0|      0.0|      0.0|       0.0|  0.0|        1.0|       1.0|          0.0|02ef2e04-5a06-4f1...|52b4e8e3-4481-44e...|02ef2e04-5a06-4f1...|
|2023-09-19|   \|     PM|e42a0825-379b-449...|   0.0|      1.0|        0.0|      0.0|      0.0|       0.0|  0.0|        1.0|       1.0|          0.0

In [14]:
show_df_missing_breakdown(df_attendancesessions_spark)

DataFrame has 16311626 rows and 17 columns.

Column                             Null  EmptyStr  NA/NaNStr  NumericNaN  TotalMissing  %Missing
----------------------------------------------------------------------
Date                                  0         0         0           0            0      0.00%
Mark                                  0         0         0           0            0      0.00%
Session                               0         0         0           0            0      0.00%
attendancesessionkey                  0         0         0           0            0      0.00%
is_aea                              506         0         0           0          506      0.00%
is_attend                           506         0         0           0          506      0.00%
is_auth_abs                         506         0         0           0          506      0.00%
is_late_L                           506         0         0           0          506      0.00%
is_late_U          

Initial observations:
- Df has 16,000,000 rows! 
- 3 keys: student_id, organisation_id, date
- date can be used to join with the date dimension - via new column datekey

In [15]:
from pyspark.sql import functions as F

#Creat a new column datekey in the df_attendancesessions_spark which will take the value of the Date column without the "-" character.
#This can act as key to join the df_attendancesessions_spark with the df_date_spark

df_attendancesessions_spark = df_attendancesessions_spark.withColumn(
    "datekey",
    F.regexp_replace("Date", "-", "").cast("int")
)

In [16]:
df_attendancesessions_spark.show() #check the new column datekey



+----------+----+-------+--------------------+------+---------+-----------+---------+---------+----------+-----+-----------+----------+-------------+--------------------+--------------------+--------------------+--------+
|      Date|Mark|Session|attendancesessionkey|is_aea|is_attend|is_auth_abs|is_late_L|is_late_U|is_missing|is_nr|is_possible|is_present|is_unauth_abs|     organisationkey|          studentkey|        partitionkey| datekey|
+----------+----+-------+--------------------+------+---------+-----------+---------+---------+----------+-----+-----------+----------+-------------+--------------------+--------------------+--------------------+--------+
|2023-11-13|   /|     AM|162ff625-c9d6-49e...|   0.0|      1.0|        0.0|      0.0|      0.0|       0.0|  0.0|        1.0|       1.0|          0.0|02ef2e04-5a06-4f1...|52b4e8e3-4481-44e...|02ef2e04-5a06-4f1...|20231113|
|2023-09-19|   \|     PM|e42a0825-379b-449...|   0.0|      1.0|        0.0|      0.0|      0.0|       0.0|  0.0|

In [17]:
df_date_spark.show() #check the df_date_spark

+--------+----------+-----------------+--------------------+--------------------+-------------------+------------------+----------------------------------+---------------------------------+-------------------+---------------+-------------+---------------------+------------------+----------------+-------------------------+---------+---------------------+---------+-------------------+---------------+------------+-------------------+------------------------+-----------+------------+-------------------+-------------------+----------------+---------------+------------------+----------+-----------+-------------+-------------------+-------------+-------------+--------+----------+----------+----------+-----------+-------------+----------------+---------+---------+---------+----------------+-------------+----------------+------------------+-------------+-------------------------+------------------+-----------------+-----------------------+
| DateKey|  FullDate|MonthNumberOfYear|MonthNumberOfQua

In [18]:
show_df_missing_breakdown(df_date_spark)

DataFrame has 36891 rows and 56 columns.

Column                             Null  EmptyStr  NA/NaNStr  NumericNaN  TotalMissing  %Missing
----------------------------------------------------------------------
DateKey                               0         0         0           0            0      0.00%
FullDate                              0         0         0           0            0      0.00%
MonthNumberOfYear                     0         0         0           0            0      0.00%
MonthNumberOfQuarter                  0         0         0           0            0      0.00%
ISOYearAndWeekNumber                  0         0         0           0            0      0.00%
ISOWeekNumberOfYear                   0         0         0           0            0      0.00%
SSWeekNumberOfYear                    0         0         0           0            0      0.00%
ISOWeekNumberOfQuarter_454_Pattern    0         0         0           0            0      0.00%
SSWeekNumberOfQuarter_

Initial observations:
- Academic year is available in the data - good for academic year analysis
- week start dates available as calendar week start date and academic week start date
- datekey can be used to join with the date dimension

#### 4. Join the data

In [19]:
from pyspark.sql import functions as F

# 1. Alias DataFrames to reference them in the join condition and in the column selection 
df_att_aliased = df_attendancesessions_spark.alias("att")
df_org_aliased = df_organisation_spark.alias("org")
df_stu_aliased = df_student_spark.alias("stu")
df_stex_aliased = df_studentextended_spark.alias("stex")
df_date_aliased = df_date_spark.alias("dd")

# 2. Join them explicitly
df_joined = (
    df_att_aliased
    .join(df_org_aliased, df_att_aliased["organisationkey"] == df_org_aliased["organisationkey"], "left")
    .join(df_stu_aliased, df_att_aliased["studentkey"] == df_stu_aliased["studentkey"], "left")
    .join(df_stex_aliased, df_att_aliased["studentkey"] == df_stex_aliased["studentkey"], "left")
    .join(df_date_aliased, df_att_aliased["datekey"] == df_date_aliased["DateKey"], "left") #join the df_attendancesessions_spark with the df_date_spark
)

# 3. Programmatically build a list of columns to select
#    Each column is referenced by alias + column name, and renamed with a prefix
att_cols = [F.col(f"att.{c}").alias(f"att_{c}") for c in df_attendancesessions_spark.columns]
org_cols = [F.col(f"org.{c}").alias(f"org_{c}") for c in df_organisation_spark.columns]
stu_cols = [F.col(f"stu.{c}").alias(f"stu_{c}") for c in df_student_spark.columns]
stex_cols = [F.col(f"stex.{c}").alias(f"stex_{c}") for c in df_studentextended_spark.columns]
date_cols = [F.col(f"dd.{c}").alias(f"dd_{c}") for c in df_date_spark.columns]

# Combine all these column lists
all_cols = att_cols + org_cols + stu_cols + stex_cols + date_cols

# 4. Select everything into a new DataFrame, with prefixed column names
df_joined_renamed = df_joined.select(*all_cols)

df_joined_renamed.show(truncate=False)


+----------+--------+-----------+------------------------------------+----------+-------------+---------------+-------------+-------------+--------------+---------+---------------+--------------+-----------------+------------------------------------+------------------------------------+------------------------------------+-----------+---------------------+------------------------+-----------+---------------------+------------------------------------+--------------+---------+-----------------------+----------------+-------+------------------------------------+------------+------------------+-----------------+-----------+----------------+-------+----------+-----------------+------------------------------------+------------------------------------+----------+------------------------------------+--------------------------+-------------------+-----------------+-------------------+----------------------+------------------------+-------------------------------+----------------+---------------

Check: First row of dates match in all columns 2022-01-03, so I will use this date to calculate the week number.

In [20]:
#check the data types of the columns
df_joined_renamed.dtypes 

[('att_Date', 'string'),
 ('att_Mark', 'string'),
 ('att_Session', 'string'),
 ('att_attendancesessionkey', 'string'),
 ('att_is_aea', 'double'),
 ('att_is_attend', 'double'),
 ('att_is_auth_abs', 'double'),
 ('att_is_late_L', 'double'),
 ('att_is_late_U', 'double'),
 ('att_is_missing', 'double'),
 ('att_is_nr', 'double'),
 ('att_is_possible', 'double'),
 ('att_is_present', 'double'),
 ('att_is_unauth_abs', 'double'),
 ('att_organisationkey', 'string'),
 ('att_studentkey', 'string'),
 ('att_partitionkey', 'string'),
 ('att_datekey', 'int'),
 ('org_Organisation_Name', 'string'),
 ('org_Establishment_Number', 'string'),
 ('org_LA_Code', 'string'),
 ('org_Organisation_Type', 'string'),
 ('org_organisationkey', 'string'),
 ('org_addresskey', 'string'),
 ('org_UKPRN', 'string'),
 ('org_Organisation_Status', 'string'),
 ('org_last_updated', 'string'),
 ('org_URN', 'bigint'),
 ('org_partitionkey', 'string'),
 ('stu_Forename', 'string'),
 ('stu_Legal_Forename', 'string'),
 ('stu_Legal_Surname'

In [21]:


total_rows = df_joined_renamed.count()
print(f"Total rows: {total_rows}")     

Total rows: 16311626


In [22]:
column_count = len(df_joined_renamed.columns)
print(f"Number of columns: {column_count}")


Number of columns: 122



The following is a list of columns that can potentially be useful when creating an attendance report. I have consulted the **Department for Education (DfE)** for definitions, as they use specific data fields to collect and manage information related to students and educational establishments. This can also be used to determine a suitable alias for each field. Below is an explanation of each field in the dataset:

1. **student_sex**: Indicates the student's gender, typically recorded as 'M' for male or 'F' for female.  
2. **student_forename**: The student's first name.  
3. **organisation_type**: Specifies the type of educational establishment, such as 'Academy', 'Community School', 'Free School', etc.  
4. **organisation_name**: The official name of the educational establishment.  
5. **establishment_number**: A unique 4-digit number assigned to each educational establishment by the DfE. This number, combined with the local authority number, forms the DfE number used to identify schools.  
6. **la_code**: The Local Authority code, a 3-digit number representing the local authority responsible for the educational establishment. This code, combined with the establishment number, forms the DfE number.  
7. **attendance_date**: The specific date for which a student's attendance is recorded.  
8. **mark**: The attendance code indicating a student's presence or type of absence for a particular session. The DfE provides a set of standardised attendance codes to describe pupil attendance and absence.  
9. **session**: Denotes whether the attendance record pertains to the morning (AM) or afternoon (PM) session of the school day.  
10. **is_aea**: Indicates whether the session is an Approved Educational Activity (AEA), meaning the student is off-site but engaged in supervised educational activities approved by the school.  
11. **is_attend**: Specifies if the student attended the session.  
12. **is_auth_abs**: Indicates if the student's absence for the session was authorised by the school.  
13. **is_late_L**: Shows if the student arrived late to the session but before the register closed, typically marked with code 'L'.  
14. **is_late_U**: Indicates if the student arrived after the register closed, usually marked with code 'U', which can denote an unauthorised absence.  
15. **is_missing**: Denotes if the attendance data for the session is missing or not recorded.  
16. **is_nr**: Indicates 'No Reason' provided for absence, showing that no explanation has been given for the student's absence.  
17. **is_possible**: Specifies if the session was a possible attendance session for the student, meaning they were expected to attend.  
18. **is_present**: Indicates if the student was present during the session.  
19. **is_unauth_abs**: Shows if the student's absence was unauthorised.  
20. **UPN**: Unique Pupil Number, a 13-character identifier assigned to each student in England to track their educational progress.  
21. **academic_year**: The academic year to which the data pertains, typically spanning from September of one year to August of the next (e.g., 2024/2025).  
22. **week_number**: The specific week of the academic year, often numbered from 1 onwards, starting from the beginning of the school year.  
23. **term_session**: Indicates the term (e.g., Autumn, Spring, Summer) and the specific session within that term.  



In [23]:
from pyspark.sql import functions as F

df_selected = (
    df_joined_renamed
    .select(
        # --- Student details & school info ---
        F.col("stu_Sex").alias("gender"),
        F.col("stu_Forename").alias("student_forename"),
        F.col("stu_Surname").alias("student_surname"),
        F.col("stex_Pupil_Premium_Indicator").alias("pupil_premium"),
        F.col("stex_Year_Group").alias("year_group"),
        F.col("stex_Current_NC_Year").alias("nc_year"),
        F.col("org_Organisation_Type").alias("school_type"),
        F.col("org_Organisation_Name").alias("school"),
        F.col("org_Establishment_Number").alias("establishment_number"),
        F.col("org_LA_Code").alias("la_code"),

        # --- Dates ---
        F.col("att_Date").alias("attendance_date"),
        F.col("dd_AcademicYear").alias("academic_year"),
        F.col("dd_AcademicWeekNumberOfYear").alias("academic_week_number"),
        F.col("dd_TermSession").alias("term"),
        # Replace below if "weekcommencingdate" doesn't exist. 
        # For example, use "dd_WeekStartDate" or "dd_WeekCommencing DD/MM/YYYY" from your schema.
        F.col("dd_WeekCommencingName").alias("weekcommencing"),

        # --- Attendance fields ---
        F.col("att_Mark").alias("mark"),
        F.col("att_Session").alias("session"),
        # Use a valid alias for 'att_is_aea' (spaces in column names can cause issues)
        F.col("att_is_aea").alias("is_approved_educational_activity"),
        F.col("att_is_attend").alias("is_attend"),
        F.col("att_is_auth_abs").alias("is_auth_abs"),
        F.col("att_is_late_L").alias("late"),
        F.col("att_is_late_U").alias("late_unauthorised"),
        F.col("att_is_missing").alias("missing"),
        F.col("att_is_nr").alias("no_reason"),
        F.col("att_is_possible").alias("is_possible"),
        F.col("att_is_present").alias("is_present"),
        F.col("att_is_unauth_abs").alias("is_unauth_abs"),

        # --- Current student info ---
        F.col("stex_Is_Current").alias("current_student"),
        F.col("stex_Leaving_Date").alias("leaving_date"),
        F.col("stu_UPN").alias("UPN")
    )
)

df_selected.show(truncate=False)


+------+----------------+---------------+-------------+----------+-------+-----------+---------+--------------------+-------+---------------+-------------+--------------------+------+--------------+----+-------+--------------------------------+---------+-----------+----+-----------------+-------+---------+-----------+----------+-------------+---------------+------------+----+
|gender|student_forename|student_surname|pupil_premium|year_group|nc_year|school_type|school   |establishment_number|la_code|attendance_date|academic_year|academic_week_number|term  |weekcommencing|mark|session|is_approved_educational_activity|is_attend|is_auth_abs|late|late_unauthorised|missing|no_reason|is_possible|is_present|is_unauth_abs|current_student|leaving_date|UPN |
+------+----------------+---------------+-------------+----------+-------+-----------+---------+--------------------+-------+---------------+-------------+--------------------+------+--------------+----+-------+-------------------------------

#### 5. Inspect Distinct Values

Begin by looking at year group as it is a key field needed in the summary table.

In [24]:
df_selected.select("year_group").distinct().show()

+----------+
|year_group|
+----------+
|         7|
|        11|
|         8|
|         9|
|        10|
|        12|
|        13|
|         3|
|         5|
| Nursery 2|
|         6|
|         R|
| Nursery 1|
|         1|
|         4|
|         2|
|   Year 13|
|       Y10|
|       Y08|
|       Y07|
+----------+
only showing top 20 rows



Using these values with skew the data, as Y07, 7 point to the same year group. 

In [25]:
df_tidy = (
    df_selected
    .withColumn(
        "year_group_tidy",
        F.when(
            F.col("year_group").isin("Nursery 1", "Nursery 2"),
            F.regexp_replace("year_group", "Nursery ", "N")
        )
        .when(
            F.col("year_group") == "R",
            F.lit("Reception")
        )
        .when(
            F.col("year_group") == "Year 13",
            F.lit("Y13")
        )
        .when(
            F.col("year_group").rlike("^[0-9]+$"),
            F.concat(F.lit("Y"), F.col("year_group").cast("int"))
        )
        .when(
            F.col("year_group").rlike("^Y[0-9]{1,2}$"),
            F.concat(F.lit("Y"), F.regexp_replace("year_group", "^[Yy]", "").cast("int"))
        )
        .otherwise(F.col("year_group"))
    )
)

# Get distinct values
distinct_vals = df_tidy.select("year_group_tidy").distinct()
count_distinct = distinct_vals.count()

print(f"Number of distinct values in 'year_group_tidy': {count_distinct}")
distinct_vals.show(truncate=False)


Number of distinct values in 'year_group_tidy': 18
+---------------+
|year_group_tidy|
+---------------+
|Y10            |
|Y12            |
|Y11            |
|Y13            |
|Y8             |
|Y7             |
|Y9             |
|Y6             |
|Y2             |
|Reception      |
|Y4             |
|Y3             |
|Y1             |
|N2             |
|N1             |
|Y5             |
|Y14            |
|NULL           |
+---------------+



In [26]:
show_df_missing_breakdown(df_tidy.select("year_group_tidy"))

DataFrame has 16311626 rows and 1 columns.

Column                             Null  EmptyStr  NA/NaNStr  NumericNaN  TotalMissing  %Missing
----------------------------------------------------------------------
year_group_tidy                   43632         0         0           0        43632      0.27%


In [27]:
#make a count of the number of students in each year group
df_tidy.groupBy("year_group_tidy").count().show()   

+---------------+-------+
|year_group_tidy|  count|
+---------------+-------+
|            Y10|3986310|
|           NULL|  43632|
|            Y12| 789139|
|            Y11|4008659|
|            Y13| 811876|
|             Y8|1919106|
|             Y7| 581063|
|             Y9|3226089|
|             Y6| 188560|
|             Y2| 104254|
|      Reception|  35382|
|             Y4| 160217|
|             Y3| 154381|
|             Y1|  80604|
|             N2|  13913|
|             N1|   1466|
|             Y5| 197483|
|            Y14|   9492|
+---------------+-------+



In [28]:
df_selected.select("nc_year").distinct().show()

+-------+
|nc_year|
+-------+
|      7|
|     11|
|      8|
|      9|
|     10|
|     12|
|     13|
|      3|
|      5|
|      6|
|      R|
|      1|
|     N2|
|      4|
|     N1|
|      2|
+-------+



In [29]:
#count the number of students in each NC year
df_selected.groupBy("nc_year").count().show()

+-------+-------+
|nc_year|  count|
+-------+-------+
|      7| 588774|
|     11|4017479|
|      8|1924729|
|      9|3235343|
|     10|3994364|
|     12| 789109|
|     13| 824412|
|      3| 154541|
|      5| 197939|
|      6| 188708|
|      R|  35382|
|      1|  80604|
|     N2|  13989|
|      4| 160533|
|     N1|   1466|
|      2| 104254|
+-------+-------+



In [30]:
show_df_missing_breakdown(df_selected.select("nc_year"))

DataFrame has 16311626 rows and 1 columns.

Column                             Null  EmptyStr  NA/NaNStr  NumericNaN  TotalMissing  %Missing
----------------------------------------------------------------------
nc_year                               0         0         0           0            0      0.00%


The **National Curriculum Year** is the most accurate representation of the year group, as it is based on the student's age. It is also less prone to errors, as it is not manually entered like the **year group** field. Additionally, there are no missing values in the **National Curriculum Year** field. Therefore, I will use this field instead of the **year group** field.

In [31]:
# number of distinct values in the columns all except 'student_forename','student_surname', and 'UPN'

show_distinct_counts_approx(df_selected.drop("student_forename", "attendance_date", "student_surname", "UPN"))



Column                             Approx Distinct Count
------------------------------------------------------------
weekcommencing                     183
leaving_date                       74
academic_week_number               54
mark                               49
year_group                         31
nc_year                            16
school                             9
term                               6
academic_year                      5
gender                             2
pupil_premium                      2
school_type                        2
session                            2
is_approved_educational_activity   2
is_attend                          2
is_auth_abs                        2
late                               2
late_unauthorised                  2
missing                            2
no_reason                          2

All Column Approx Distinct Counts (showing top_n only):


Py4JJavaError: An error occurred while calling o4093.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 138.0 failed 1 times, most recent failure: Lost task 0.0 in stage 138.0 (TID 303) (Saqib executor driver): java.net.SocketException: Connection reset
	at java.net.SocketInputStream.read(Unknown Source)
	at java.net.SocketInputStream.read(Unknown Source)
	at java.io.BufferedInputStream.fill(Unknown Source)
	at java.io.BufferedInputStream.read(Unknown Source)
	at java.io.DataInputStream.readInt(Unknown Source)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:774)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at scala.collect

In [33]:
#list the distinct values in the column 'mark'
df_selected.select("mark").distinct().show()
df_selected.select("academic_year").distinct().show()


+----+
|mark|
+----+
|   K|
| I01|
| I02|
|   E|
|   B|
|   Y|
| X05|
|   L|
|   M|
|   V|
|   U|
|   O|
|   D|
|  Y6|
|   C|
|   J|
|   -|
|   Z|
| X06|
|   /|
+----+
only showing top 20 rows

+-------------+
|academic_year|
+-------------+
|    2021/2022|
|    2024/2025|
|    2022/2023|
|    2023/2024|
|    2020/2021|
+-------------+




- The **Mark** category may require further investigation by the data analyst to identify rows where the attendance data is ambiguous.  
- Additionally, the **weekcommencing** column contains 181 unique values, whereas the **academic_week_number** column contains 54 unique values. This discrepancy is expected, as the **academic_week_number** is based on the academic year and repeats across years, while **weekcommencing** is based on actual dates. The **weekcommencing** field may be more suitable for categorising the data based on week numbers.


In [34]:
df_selected.show(truncate=False)

+------+----------------+---------------+-------------+----------+-------+-----------+---------+--------------------+-------+---------------+-------------+--------------------+------+--------------+----+-------+--------------------------------+---------+-----------+----+-----------------+-------+---------+-----------+----------+-------------+---------------+------------+----+
|gender|student_forename|student_surname|pupil_premium|year_group|nc_year|school_type|school   |establishment_number|la_code|attendance_date|academic_year|academic_week_number|term  |weekcommencing|mark|session|is_approved_educational_activity|is_attend|is_auth_abs|late|late_unauthorised|missing|no_reason|is_possible|is_present|is_unauth_abs|current_student|leaving_date|UPN |
+------+----------------+---------------+-------------+----------+-------+-----------+---------+--------------------+-------+---------------+-------------+--------------------+------+--------------+----+-------+-------------------------------

#### 6. Rationale for Selecting Certain Columns:

- **Names and UPN**: These are essential for identifying students and their attendance records, particularly if a report needs to deep dive into individual student attendance. They can also help identify students with chronic absenteeism or other attendance-related issues and ensure the quality of the data.  
- **Year group**: This column is used to calculate the attendance percentage for each school on a weekly basis by year group.  
- **Establishment name and number**: These are crucial for identifying the school and its location, which can be useful for comparing attendance rates across different schools or regions. Although this dataset anonymises these fields (indicated by XXXX), I am assuming they would be provided in a real-world scenario for more accurate analysis.  
- **Attendance date, mark, and session**: These columns are essential for tracking student attendance on a daily basis and identifying patterns of absence or lateness. They will also be used later to create a unique identifier for each attendance record.  
- **Detailed attendance status columns**:  
  - **is_present**, **is_possible**, **is_auth_abs**, **is_unauth_abs**, **is_late_L**, **is_late_U**, **is_missing**, **is_nr**, and **is_aea**: These provide detailed information about the student’s attendance status, including whether the absence was authorised, unauthorised, or due to other reasons. This information can help identify trends in attendance and inform interventions to improve attendance rates.  
- **Academic year, week number, and term_session**: These are necessary for aggregating attendance data over specific time periods (e.g., weekly, termly) and tracking trends across the academic year. This information can highlight seasonal patterns in attendance and guide targeted interventions.  
- **Week commencing**: Provided in two formats to allow flexibility in reporting and analysis.



#### 7. Data Integrity Check

I create a unique identifier for each attendance record by concatenating the student_id (UPN), date, session (AM/PM) columns. This identifier will be used to check for duplicate records and ensure data integrity.

In [35]:


# 1. Create a new field by concatenating UPN and attendance_date and session (AM/PM) with an underscore separator   
#this will be used to identify the unique attendance record for each student per day per session    
df_with_combined = df_selected.withColumn(
    "UPN_AttendanceDate",
    F.concat_ws("_", F.col("UPN"), F.col("attendance_date"), F.col("session"))
)


# 2. Group by this new field and filter for count == 1 (i.e. unique)
df_valid = (
    df_with_combined
    .groupBy("UPN_AttendanceDate")
    .count()
    .filter(F.col("count") == 1) #each student can have only one attendance per day per session AM or PM
)

# 3. Group by this new field and filter for count > 1 (i.e. duplicates)
df_invalid = (
    df_with_combined
    .groupBy("UPN_AttendanceDate")
    .count()
    .filter(F.col("count") > 1)
)


df_with_combined.show(truncate=False)


+------+----------------+---------------+-------------+----------+-------+-----------+---------+--------------------+-------+---------------+-------------+--------------------+------+--------------+----+-------+--------------------------------+---------+-----------+----+-----------------+-------+---------+-----------+----------+-------------+---------------+------------+----+------------------+
|gender|student_forename|student_surname|pupil_premium|year_group|nc_year|school_type|school   |establishment_number|la_code|attendance_date|academic_year|academic_week_number|term  |weekcommencing|mark|session|is_approved_educational_activity|is_attend|is_auth_abs|late|late_unauthorised|missing|no_reason|is_possible|is_present|is_unauth_abs|current_student|leaving_date|UPN |UPN_AttendanceDate|
+------+----------------+---------------+-------------+----------+-------+-----------+---------+--------------------+-------+---------------+-------------+--------------------+------+--------------+----+-

In [36]:
df_invalid.show(truncate=False)

+------------------+-----+
|UPN_AttendanceDate|count|
+------------------+-----+
|5798_2024-08-27_PM|2    |
|1052_2024-08-09_AM|2    |
|4878_2024-09-14_AM|2    |
|8152_2022-07-21_PM|2    |
|8152_2022-09-18_PM|2    |
|8152_2023-12-25_PM|2    |
|8152_2022-11-27_PM|2    |
|8152_2022-07-02_AM|2    |
|8152_2022-07-22_PM|2    |
|8152_2024-04-07_AM|2    |
|2584_2024-10-04_PM|2    |
|9403_2024-08-14_PM|2    |
|5008_2022-07-12_AM|2    |
|5008_2022-07-27_AM|2    |
|5008_2022-05-21_AM|2    |
|5008_2022-04-18_AM|2    |
|3924_2024-08-26_AM|2    |
|2899_2024-09-08_AM|2    |
|3898_2024-07-07_PM|2    |
|3898_2024-06-25_PM|2    |
+------------------+-----+
only showing top 20 rows



In [37]:
print(f"Total rows with original df: {df_joined_renamed.count()}") 

print(f"Total rows with selected data df: {df_selected.count()}") 

print(f"Total rows valid data: {df_valid.count()}")

print(f"Total rows invalid data: {df_invalid.count()}")

Total rows with original df: 16311626
Total rows with selected data df: 16311626
Total rows valid data: 15983332
Total rows invalid data: 164114


So there are 164114 duplicate rows. I will label these as invalid for the data analyst.
So as to not prevent any data leakage and to be able to filter easily, I will add a column called 'status' and 'count' and label each row with either valid if it occurs once, and if more than once then 'invalid' 

In [38]:


# 1. Aggregate all keys with their counts
df_counts = (
    df_with_combined
    .groupBy("UPN_AttendanceDate")
    .agg(F.count("*").alias("count"))
    .withColumn(
        "status", 
        F.when(F.col("count") == 1, "valid").otherwise("invalid")
    )
)

# 2. Join back to original rows to get the full data plus the status
df_with_status = (
    df_with_combined.alias("a")
    .join(df_counts.alias("b"), on="UPN_AttendanceDate", how="left")
    .select("a.*", "b.count", "b.status")
)

df_with_status.show(truncate=False)


+------------------------+------+----------------+---------------+-------------+----------+-------+-----------+---------+--------------------+-------+---------------+-------------+--------------------+------+--------------+----+-------+--------------------------------+---------+-----------+----+-----------------+-------+---------+-----------+----------+-------------+---------------+------------+----------+-----+-------+
|UPN_AttendanceDate      |gender|student_forename|student_surname|pupil_premium|year_group|nc_year|school_type|school   |establishment_number|la_code|attendance_date|academic_year|academic_week_number|term  |weekcommencing|mark|session|is_approved_educational_activity|is_attend|is_auth_abs|late|late_unauthorised|missing|no_reason|is_possible|is_present|is_unauth_abs|current_student|leaving_date|UPN       |count|status |
+------------------------+------+----------------+---------------+-------------+----------+-------+-----------+---------+--------------------+-------+--

In [39]:
# the valid and invalid rows can be filtered as below and called upon as needed
df_validf_rows = df_with_status.filter("status == 'valid'")
df_invalidf_rows = df_with_status.filter("status == 'invalid'")


In [40]:
#cound the number of rows in the df_valid_rows
print(f"Total rows valid data: {df_validf_rows.count()}")

#cound the number of rows in the df_invalid_rows
print(f"Total rows invalid data: {df_invalidf_rows.count()}")

Total rows valid data: 15983332
Total rows invalid data: 328294


The invalid data is approximately twice that of the previous count, as now it is counting each instance where as before it was counting the grouping of the data.

#### 8. Summary Table



Going forward, I will use the following to filter rows based on their status:

- `df_valid = df[df['status'] == 'valid']` to filter out the valid rows.
- `df_invalid_rows = df[df['status'] == 'invalid']` to filter out the invalid rows, which can be used to investigate duplicates further if needed.

I will create an initial summary table that contains the attendance percentage for each school on a weekly basis by the **Year Group** of the student. The table will include the following columns:

- **School**: The name of the school.  
- **week_number**: The specific week based on the week commencing date.  
- **year_group**: The year group of the student based on the National Curriculum Year.  
- **attendance_percentage**: The percentage of attendance for the school in the specified week and year group.  

The summary will be sorted based on the school name, week number, and year group as per the data analyst's request.

I will then print summary statistics for this data, including the **mean**, **median**, **minimum**, **maximum**, and **standard deviation** of the attendance percentage across all schools, weeks, and year groups.

Finally, I will write the summary table to `fact_AttendanceSummary` in **Parquet** format for further analysis and reporting by the Data Analyst.



In [41]:
df_summary = (
    df_validf_rows
    .groupBy("school",  "nc_year", "weekcommencing")
    .agg(
        F.round(
            (F.sum("is_attend") / F.sum("is_possible") * 100), 1
        ).alias("attendance_percentage")
    )
)

#df_summary.limit(20).show(truncate=False)

In [42]:
df_summary.show(truncate=False)

+---------+-------+--------------+---------------------+
|school   |nc_year|weekcommencing|attendance_percentage|
+---------+-------+--------------+---------------------+
|Academy 8|13     |w/c 04/09/2023|99.4                 |
|Academy 7|13     |w/c 28/02/2022|98.2                 |
|Academy 9|3      |w/c 28/11/2022|93.1                 |
|Academy 8|11     |w/c 21/03/2022|93.8                 |
|Academy 8|11     |w/c 15/04/2024|92.9                 |
|Academy 7|9      |w/c 21/08/2023|NULL                 |
|Academy 2|13     |w/c 04/03/2024|95.9                 |
|Academy 3|8      |w/c 16/10/2023|96.3                 |
|Academy 8|11     |w/c 01/08/2022|NULL                 |
|Academy 4|9      |w/c 15/05/2023|92.7                 |
|Academy 7|7      |w/c 18/11/2024|92.7                 |
|Academy 5|13     |w/c 20/02/2023|97.2                 |
|Academy 5|13     |w/c 11/03/2024|93.6                 |
|Academy 4|13     |w/c 15/11/2021|94.2                 |
|Academy 9|9      |w/c 12/09/20

In [43]:
df_summary.limit(20).show(truncate=False)

+---------+-------+--------------+---------------------+
|school   |nc_year|weekcommencing|attendance_percentage|
+---------+-------+--------------+---------------------+
|Academy 8|13     |w/c 04/09/2023|99.4                 |
|Academy 7|13     |w/c 28/02/2022|98.2                 |
|Academy 9|3      |w/c 28/11/2022|93.1                 |
|Academy 8|11     |w/c 21/03/2022|93.8                 |
|Academy 8|11     |w/c 15/04/2024|92.9                 |
|Academy 7|9      |w/c 21/08/2023|NULL                 |
|Academy 2|13     |w/c 04/03/2024|95.9                 |
|Academy 3|8      |w/c 16/10/2023|96.3                 |
|Academy 8|11     |w/c 01/08/2022|NULL                 |
|Academy 4|9      |w/c 15/05/2023|92.7                 |
|Academy 7|7      |w/c 18/11/2024|92.7                 |
|Academy 5|13     |w/c 20/02/2023|97.2                 |
|Academy 5|13     |w/c 11/03/2024|93.6                 |
|Academy 4|13     |w/c 15/11/2021|94.2                 |
|Academy 9|9      |w/c 12/09/20

#### 9. Null Values to be Investigated Further 

As expected, a number of NULL values are present in the data. For the attention of the data analyst, I will quantify the number of NULL values in the attendance percentage column and provide this information in the summary statistics.


In [44]:


cols = df_summary.columns

# Build a filter condition: (col1 IS NULL) OR (col2 IS NULL) OR ...
null_condition = reduce(lambda acc, c: acc | F.col(c).isNull(), cols, F.lit(False))

# Filter rows where any column is null
num_rows_with_null = df_summary.filter(null_condition).count()

print(f"Number of rows with at least one NULL value: {num_rows_with_null}")

Number of rows with at least one NULL value: 2233


I will create a table to show the number of null values in the attendance columns. This will help to identify any patterns or trends in the missing data and inform data cleaning and imputation strategies.

In [45]:
show_df_missing_breakdown(df_summary)

DataFrame has 8697 rows and 4 columns.

Column                             Null  EmptyStr  NA/NaNStr  NumericNaN  TotalMissing  %Missing
----------------------------------------------------------------------
school                                0         0         0           0            0      0.00%
nc_year                               0         0         0           0            0      0.00%
weekcommencing                        0         0         0           0            0      0.00%
attendance_percentage              2233         0         0           0         2233     25.68%


In [46]:

# 1) From df_selected, group by the same columns used in df_summary
df_sums = (
    df_validf_rows
        .groupBy("school",  "nc_year", "weekcommencing")
        .agg(
            F.sum("is_attend").alias("sum_is_attend"),
            F.sum("is_possible").alias("sum_is_possible")
        )
)


# 2) Filter df_summary for rows where attendance_percentage is NULL
df_summary_nulls = df_summary.filter(F.col("attendance_percentage").isNull())

# 3) Join df_summary_nulls with df_sums to see actual sums for those groups
df_null_sums = (
    df_summary_nulls.alias("summ")
    .join(
        df_sums.alias("sums"),
        on=["school", "nc_year", "weekcommencing"],
        how="left"
    )
    .select(
        "summ.school",
        "summ.nc_year",
        "summ.weekcommencing",
        "summ.attendance_percentage",   # should be NULL
        "sums.sum_is_attend",
        "sums.sum_is_possible"
    )
)

df_null_sums.show(truncate=False)


+---------+-------+--------------+---------------------+-------------+---------------+
|school   |nc_year|weekcommencing|attendance_percentage|sum_is_attend|sum_is_possible|
+---------+-------+--------------+---------------------+-------------+---------------+
|Academy 7|9      |w/c 21/08/2023|NULL                 |0.0          |0.0            |
|Academy 8|11     |w/c 01/08/2022|NULL                 |0.0          |0.0            |
|Academy 9|11     |w/c 28/08/2023|NULL                 |0.0          |0.0            |
|Academy 9|11     |w/c 12/08/2024|NULL                 |0.0          |0.0            |
|Academy 1|10     |w/c 12/02/2024|NULL                 |0.0          |0.0            |
|Academy 6|11     |w/c 07/08/2023|NULL                 |0.0          |0.0            |
|Academy 9|5      |w/c 01/04/2024|NULL                 |0.0          |0.0            |
|Academy 6|10     |w/c 27/12/2021|NULL                 |0.0          |0.0            |
|Academy 8|11     |w/c 23/08/2021|NULL     

To drill down further I can use the df_validf_rows dataframe to investigate the missing data further. This contains only the valid rows before any summary data frame, so I can see if there is a pattern in the missing data.

In [47]:
df_validf_rows.filter(
  (F.col("school") == "Academy 7") &
  (F.col("nc_year") == "9") & 
  (F.col("weekcommencing") == "w/c 21/08/2023")

).show()


+-------------------+------+----------------+---------------+-------------+----------+-------+-----------+---------+--------------------+-------+---------------+-------------+--------------------+------+--------------+----+-------+--------------------------------+---------+-----------+----+-----------------+-------+---------+-----------+----------+-------------+---------------+------------+-----+-----+------+
| UPN_AttendanceDate|gender|student_forename|student_surname|pupil_premium|year_group|nc_year|school_type|   school|establishment_number|la_code|attendance_date|academic_year|academic_week_number|  term|weekcommencing|mark|session|is_approved_educational_activity|is_attend|is_auth_abs|late|late_unauthorised|missing|no_reason|is_possible|is_present|is_unauth_abs|current_student|leaving_date|  UPN|count|status|
+-------------------+------+----------------+---------------+-------------+----------+-------+-----------+---------+--------------------+-------+---------------+-------------


The symbol for Mark = # often indicates school closure but will need further clarification. This results in the data fields is_present and is_possible both equating to zero, although the no_reason column is 1 while unauthorised absence is 0. To avoid introducing bias, I have retained these values in the dataset; the data analyst can investigate further and make adjustments as needed.

I will conclude by sorting the summary data frame by school name, week number, and year group, and writing the summary table to fact_AttendanceSummary in Parquet format for further analysis and reporting by the Data Analyst.

The summary table also includes the academic year, which, although not explicitly listed in the task brief, has been added to provide more context to the data.

#### 10. Write the Summary Table to Parquet


In [48]:

df_summary = (
    df_invalidf_rows
    
    # 1) Group and aggregate
    .groupBy("school", "nc_year", "weekcommencing")
    .agg(
        F.round(
            (F.sum("is_attend") / F.sum("is_possible") * 100), 1
        ).alias("attendance_percentage")
    )
    

)

df_summary.limit(20).show(truncate=False)



+---------+-------+--------------+---------------------+
|school   |nc_year|weekcommencing|attendance_percentage|
+---------+-------+--------------+---------------------+
|Academy 9|11     |w/c 25/03/2024|37.5                 |
|Academy 9|11     |w/c 12/08/2024|NULL                 |
|Academy 8|11     |w/c 21/03/2022|100.0                |
|Academy 8|11     |w/c 01/08/2022|NULL                 |
|Academy 3|12     |w/c 26/02/2024|100.0                |
|Academy 9|12     |w/c 23/05/2022|75.0                 |
|Academy 4|9      |w/c 15/05/2023|60.0                 |
|Academy 3|13     |w/c 17/01/2022|88.5                 |
|Academy 9|8      |w/c 22/05/2023|73.3                 |
|Academy 9|12     |w/c 12/09/2022|100.0                |
|Academy 4|11     |w/c 19/09/2022|83.9                 |
|Academy 2|12     |w/c 20/02/2023|90.0                 |
|Academy 6|10     |w/c 24/04/2023|43.8                 |
|Academy 5|13     |w/c 11/03/2024|100.0                |
|Academy 9|9      |w/c 12/09/20

In [49]:
# Basic statistics for the attendance_percentage column

from pyspark.sql.functions import round

df_summary.describe("attendance_percentage") \
    .select("summary", round("attendance_percentage", 1).alias("attendance_percentage")) \
    .show()


+-------+---------------------+
|summary|attendance_percentage|
+-------+---------------------+
|  count|               2677.0|
|   mean|                 78.0|
| stddev|                 29.4|
|    min|                  0.0|
|    max|                100.0|
+-------+---------------------+



In [50]:
#conver df_summary to pandas 
df_summary_pandas = df_summary.toPandas()

#convert pandas to parquet  
df_summary_pandas.to_parquet('data/df_summary_pandas.parquet')



In [51]:
#chceck if the file is created
df_summary_loaded = spark.read.parquet("data/df_summary_pandas.parquet")


In [52]:
df_summary_loaded.show(truncate=False)

+---------+-------+--------------+---------------------+
|school   |nc_year|weekcommencing|attendance_percentage|
+---------+-------+--------------+---------------------+
|Academy 9|11     |w/c 25/03/2024|37.5                 |
|Academy 9|11     |w/c 12/08/2024|NULL                 |
|Academy 8|11     |w/c 21/03/2022|100.0                |
|Academy 8|11     |w/c 01/08/2022|NULL                 |
|Academy 3|12     |w/c 26/02/2024|100.0                |
|Academy 9|12     |w/c 23/05/2022|75.0                 |
|Academy 4|9      |w/c 15/05/2023|60.0                 |
|Academy 3|13     |w/c 17/01/2022|88.5                 |
|Academy 9|8      |w/c 22/05/2023|73.3                 |
|Academy 9|12     |w/c 12/09/2022|100.0                |
|Academy 4|11     |w/c 19/09/2022|83.9                 |
|Academy 2|12     |w/c 20/02/2023|90.0                 |
|Academy 6|10     |w/c 24/04/2023|43.8                 |
|Academy 5|13     |w/c 11/03/2024|100.0                |
|Academy 9|9      |w/c 12/09/20

#### 11. Notes for the Data Analyst

The data has been cleaned and quality-assured to the best of my ability. I have identified and labelled duplicate records as invalid and provided a summary of the missing values in the attendance percentage column. For cases where attendance percentage = 0, the denominator and numerator for the field are often both 0; you may wish to examine how the attendance mark is recorded in the data, as this may provide further insights into the reasons for absence.

The summary table contains the attendance percentage for each school on a weekly basis by the year group of the student, sorted by school name, week number, and year group. I used the National Curriculum Year as the year group field for consistency and accuracy over school groupings. I also used the week commencing as the basis for the week number to ensure alignment with the academic year, thereby reducing the need for an additional field.

The summary statistics include the mean, median, minimum, maximum, and standard deviation of the attendance percentage across all schools, weeks, and year groups. The data has been written to fact_AttendanceSummary in Parquet format for further analysis and reporting. Please let me know if you require any additional information or further analysis.